In [4]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils


parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default = 'mnist', help='cifar10 | lsun | mnist |imagenet | folder | lfw | fake')
parser.add_argument('--dataroot', default = './data/mnist_png', help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=5, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--dry-run', action='store_true', help='check a single training cycle works')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='.', help='folder to output images and model checkpoints')

opt, unknown = parser.parse_known_args()


opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

cudnn.benchmark = True


dataset = dset.MNIST(root=opt.dataroot, download=True,
                     transform=transforms.Compose([
                     transforms.Resize(opt.imageSize),
                     transforms.ToTensor(),
                     transforms.Normalize((0.5,), (0.5,)),]))
nc=1



assert dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))

device = torch.device("cuda:0")
ngpu = int(opt.ngpu)
nz = int(opt.nz)
ngf = int(opt.ngf)
ndf = int(opt.ndf)


# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)


class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output


netG = Generator(ngpu).to(device)
netG.apply(weights_init)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG))
print(netG)


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))
print(netD)

criterion = nn.BCELoss()

fixed_noise = torch.randn(opt.batchSize, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))

if opt.dry_run:
    opt.niter = 1

for epoch in range(opt.niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label,
                           dtype=real_cpu.dtype, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, opt.niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % opt.outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (opt.outf, epoch),
                    normalize=True)

        if opt.dry_run:
            break
    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (opt.outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (opt.outf, epoch))

Random Seed:  6170
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    

[0/5][76/938] Loss_D: 0.4790 Loss_G: 18.6008 D(x): 0.8068 D(G(z)): 0.0000 / 0.0000
[0/5][77/938] Loss_D: 1.4232 Loss_G: 9.0903 D(x): 0.5040 D(G(z)): 0.0000 / 0.0007
[0/5][78/938] Loss_D: 0.6501 Loss_G: 9.7983 D(x): 0.9527 D(G(z)): 0.2758 / 0.0001
[0/5][79/938] Loss_D: 0.3863 Loss_G: 10.5486 D(x): 0.9094 D(G(z)): 0.1208 / 0.0001
[0/5][80/938] Loss_D: 0.1318 Loss_G: 8.9059 D(x): 0.9461 D(G(z)): 0.0572 / 0.0004
[0/5][81/938] Loss_D: 0.7230 Loss_G: 11.8884 D(x): 0.8380 D(G(z)): 0.2683 / 0.0000
[0/5][82/938] Loss_D: 0.3837 Loss_G: 6.8235 D(x): 0.7727 D(G(z)): 0.0107 / 0.0018
[0/5][83/938] Loss_D: 1.3205 Loss_G: 22.9077 D(x): 0.9623 D(G(z)): 0.6374 / 0.0000
[0/5][84/938] Loss_D: 3.4657 Loss_G: 12.1966 D(x): 0.1214 D(G(z)): 0.0000 / 0.0000
[0/5][85/938] Loss_D: 0.2830 Loss_G: 5.4809 D(x): 0.9993 D(G(z)): 0.1654 / 0.0072
[0/5][86/938] Loss_D: 1.8919 Loss_G: 18.5427 D(x): 0.9991 D(G(z)): 0.7594 / 0.0000
[0/5][87/938] Loss_D: 0.3184 Loss_G: 17.2890 D(x): 0.7979 D(G(z)): 0.0001 / 0.0000
[0/5][88/

[0/5][176/938] Loss_D: 0.2361 Loss_G: 4.7348 D(x): 0.9717 D(G(z)): 0.1749 / 0.0130
[0/5][177/938] Loss_D: 0.1416 Loss_G: 4.5953 D(x): 0.9150 D(G(z)): 0.0450 / 0.0149
[0/5][178/938] Loss_D: 0.1778 Loss_G: 3.4787 D(x): 0.8932 D(G(z)): 0.0482 / 0.0472
[0/5][179/938] Loss_D: 0.1587 Loss_G: 3.3763 D(x): 0.9249 D(G(z)): 0.0622 / 0.0471
[0/5][180/938] Loss_D: 0.1896 Loss_G: 4.3264 D(x): 0.9523 D(G(z)): 0.1228 / 0.0175
[0/5][181/938] Loss_D: 0.0771 Loss_G: 5.0363 D(x): 0.9634 D(G(z)): 0.0368 / 0.0097
[0/5][182/938] Loss_D: 0.1547 Loss_G: 3.5702 D(x): 0.9008 D(G(z)): 0.0301 / 0.0374
[0/5][183/938] Loss_D: 0.2034 Loss_G: 4.1207 D(x): 0.9484 D(G(z)): 0.1174 / 0.0217
[0/5][184/938] Loss_D: 0.1552 Loss_G: 3.7798 D(x): 0.9086 D(G(z)): 0.0443 / 0.0316
[0/5][185/938] Loss_D: 0.2789 Loss_G: 4.9981 D(x): 0.9279 D(G(z)): 0.1669 / 0.0096
[0/5][186/938] Loss_D: 0.3558 Loss_G: 2.2943 D(x): 0.7587 D(G(z)): 0.0230 / 0.1412
[0/5][187/938] Loss_D: 0.3642 Loss_G: 7.0608 D(x): 0.9938 D(G(z)): 0.2714 / 0.0015
[0/5

[0/5][276/938] Loss_D: 0.3725 Loss_G: 2.8960 D(x): 0.7935 D(G(z)): 0.0931 / 0.0830
[0/5][277/938] Loss_D: 0.3142 Loss_G: 3.2298 D(x): 0.8924 D(G(z)): 0.1617 / 0.0593
[0/5][278/938] Loss_D: 0.3108 Loss_G: 3.0985 D(x): 0.8495 D(G(z)): 0.1141 / 0.0624
[0/5][279/938] Loss_D: 0.2275 Loss_G: 3.3012 D(x): 0.8963 D(G(z)): 0.0993 / 0.0496
[0/5][280/938] Loss_D: 0.2521 Loss_G: 3.7792 D(x): 0.9282 D(G(z)): 0.1381 / 0.0376
[0/5][281/938] Loss_D: 0.2870 Loss_G: 3.2379 D(x): 0.8447 D(G(z)): 0.0875 / 0.0598
[0/5][282/938] Loss_D: 0.2585 Loss_G: 3.1167 D(x): 0.8862 D(G(z)): 0.1100 / 0.0645
[0/5][283/938] Loss_D: 0.2558 Loss_G: 3.0986 D(x): 0.8780 D(G(z)): 0.0993 / 0.0616
[0/5][284/938] Loss_D: 0.2560 Loss_G: 3.4326 D(x): 0.9014 D(G(z)): 0.1265 / 0.0460
[0/5][285/938] Loss_D: 0.2012 Loss_G: 3.8363 D(x): 0.9257 D(G(z)): 0.1065 / 0.0288
[0/5][286/938] Loss_D: 0.2145 Loss_G: 3.4060 D(x): 0.8825 D(G(z)): 0.0649 / 0.0515
[0/5][287/938] Loss_D: 0.1989 Loss_G: 3.3574 D(x): 0.9420 D(G(z)): 0.1229 / 0.0470
[0/5

[0/5][376/938] Loss_D: 0.1364 Loss_G: 3.7010 D(x): 0.9508 D(G(z)): 0.0762 / 0.0345
[0/5][377/938] Loss_D: 0.1593 Loss_G: 3.1852 D(x): 0.9172 D(G(z)): 0.0640 / 0.0546
[0/5][378/938] Loss_D: 0.1735 Loss_G: 4.2270 D(x): 0.9505 D(G(z)): 0.1048 / 0.0229
[0/5][379/938] Loss_D: 0.1427 Loss_G: 3.8859 D(x): 0.9279 D(G(z)): 0.0622 / 0.0299
[0/5][380/938] Loss_D: 0.2035 Loss_G: 2.3933 D(x): 0.8623 D(G(z)): 0.0411 / 0.1270
[0/5][381/938] Loss_D: 0.2727 Loss_G: 5.0443 D(x): 0.9637 D(G(z)): 0.1887 / 0.0099
[0/5][382/938] Loss_D: 0.2324 Loss_G: 2.7843 D(x): 0.8265 D(G(z)): 0.0197 / 0.0777
[0/5][383/938] Loss_D: 0.1237 Loss_G: 3.3305 D(x): 0.9643 D(G(z)): 0.0783 / 0.0554
[0/5][384/938] Loss_D: 0.1543 Loss_G: 4.3056 D(x): 0.9602 D(G(z)): 0.1008 / 0.0201
[0/5][385/938] Loss_D: 0.1154 Loss_G: 4.1066 D(x): 0.9325 D(G(z)): 0.0412 / 0.0261
[0/5][386/938] Loss_D: 0.1558 Loss_G: 3.3000 D(x): 0.9202 D(G(z)): 0.0607 / 0.0566
[0/5][387/938] Loss_D: 0.1368 Loss_G: 3.6604 D(x): 0.9506 D(G(z)): 0.0746 / 0.0378
[0/5

...

[4/5][436/938] Loss_D: 0.1470 Loss_G: 5.8295 D(x): 0.9933 D(G(z)): 0.1209 / 0.0042
[4/5][437/938] Loss_D: 0.0939 Loss_G: 4.1950 D(x): 0.9225 D(G(z)): 0.0093 / 0.0197
[4/5][438/938] Loss_D: 0.0382 Loss_G: 4.2550 D(x): 0.9832 D(G(z)): 0.0202 / 0.0206
[4/5][439/938] Loss_D: 0.0478 Loss_G: 4.2220 D(x): 0.9668 D(G(z)): 0.0130 / 0.0231
[4/5][440/938] Loss_D: 0.0511 Loss_G: 4.4984 D(x): 0.9904 D(G(z)): 0.0395 / 0.0160
[4/5][441/938] Loss_D: 0.0432 Loss_G: 4.9073 D(x): 0.9873 D(G(z)): 0.0295 / 0.0106
[4/5][442/938] Loss_D: 0.0422 Loss_G: 4.8742 D(x): 0.9730 D(G(z)): 0.0137 / 0.0128
[4/5][443/938] Loss_D: 0.0363 Loss_G: 4.6810 D(x): 0.9824 D(G(z)): 0.0180 / 0.0159
[4/5][444/938] Loss_D: 0.0700 Loss_G: 4.0405 D(x): 0.9650 D(G(z)): 0.0327 / 0.0243
[4/5][445/938] Loss_D: 0.0430 Loss_G: 4.3618 D(x): 0.9787 D(G(z)): 0.0204 / 0.0182
[4/5][446/938] Loss_D: 0.0340 Loss_G: 4.7415 D(x): 0.9752 D(G(z)): 0.0084 / 0.0132
[4/5][447/938] Loss_D: 0.0397 Loss_G: 4.5932 D(x): 0.9904 D(G(z)): 0.0291 / 0.0146
[4/5

[4/5][537/938] Loss_D: 1.0499 Loss_G: 1.7291 D(x): 0.7849 D(G(z)): 0.5136 / 0.2092
[4/5][538/938] Loss_D: 0.8757 Loss_G: 1.4570 D(x): 0.5968 D(G(z)): 0.2447 / 0.2676
[4/5][539/938] Loss_D: 0.9035 Loss_G: 1.5005 D(x): 0.6889 D(G(z)): 0.3567 / 0.2592
[4/5][540/938] Loss_D: 1.2533 Loss_G: 1.5101 D(x): 0.6273 D(G(z)): 0.4918 / 0.2580
[4/5][541/938] Loss_D: 1.0285 Loss_G: 1.1547 D(x): 0.5643 D(G(z)): 0.2773 / 0.3671
[4/5][542/938] Loss_D: 0.9234 Loss_G: 1.8220 D(x): 0.7402 D(G(z)): 0.4053 / 0.1942
[4/5][543/938] Loss_D: 1.0213 Loss_G: 1.1223 D(x): 0.5432 D(G(z)): 0.2609 / 0.3615
[4/5][544/938] Loss_D: 0.8526 Loss_G: 2.0252 D(x): 0.8014 D(G(z)): 0.4200 / 0.1605
[4/5][545/938] Loss_D: 0.9977 Loss_G: 1.2221 D(x): 0.5653 D(G(z)): 0.2504 / 0.3398
[4/5][546/938] Loss_D: 0.8900 Loss_G: 1.6949 D(x): 0.7280 D(G(z)): 0.3781 / 0.2327
[4/5][547/938] Loss_D: 0.6884 Loss_G: 2.1039 D(x): 0.7561 D(G(z)): 0.3049 / 0.1404
[4/5][548/938] Loss_D: 0.7801 Loss_G: 1.3650 D(x): 0.6302 D(G(z)): 0.2180 / 0.2928
[4/5

[4/5][638/938] Loss_D: 0.4982 Loss_G: 5.6270 D(x): 0.9780 D(G(z)): 0.3314 / 0.0057
[4/5][639/938] Loss_D: 0.4423 Loss_G: 3.2859 D(x): 0.7085 D(G(z)): 0.0276 / 0.0749
[4/5][640/938] Loss_D: 0.2998 Loss_G: 3.5651 D(x): 0.9562 D(G(z)): 0.1995 / 0.0410
[4/5][641/938] Loss_D: 0.3171 Loss_G: 3.9786 D(x): 0.8916 D(G(z)): 0.1418 / 0.0315
[4/5][642/938] Loss_D: 0.1848 Loss_G: 3.5073 D(x): 0.9137 D(G(z)): 0.0816 / 0.0429
[4/5][643/938] Loss_D: 0.2892 Loss_G: 2.6441 D(x): 0.8378 D(G(z)): 0.0822 / 0.1064
[4/5][644/938] Loss_D: 0.3043 Loss_G: 4.1272 D(x): 0.9239 D(G(z)): 0.1759 / 0.0234
[4/5][645/938] Loss_D: 0.2978 Loss_G: 4.1478 D(x): 0.8884 D(G(z)): 0.1427 / 0.0275
[4/5][646/938] Loss_D: 0.4192 Loss_G: 1.8614 D(x): 0.7243 D(G(z)): 0.0425 / 0.2160
[4/5][647/938] Loss_D: 0.3308 Loss_G: 4.0707 D(x): 0.9779 D(G(z)): 0.2372 / 0.0231
[4/5][648/938] Loss_D: 0.1443 Loss_G: 4.5559 D(x): 0.9402 D(G(z)): 0.0707 / 0.0189
[4/5][649/938] Loss_D: 0.3092 Loss_G: 2.3581 D(x): 0.7877 D(G(z)): 0.0396 / 0.1361
[4/5

[4/5][739/938] Loss_D: 0.2210 Loss_G: 4.3618 D(x): 0.9269 D(G(z)): 0.1209 / 0.0198
[4/5][740/938] Loss_D: 0.2310 Loss_G: 3.3477 D(x): 0.8808 D(G(z)): 0.0786 / 0.0627
[4/5][741/938] Loss_D: 0.5697 Loss_G: 4.1177 D(x): 0.8430 D(G(z)): 0.2726 / 0.0241
[4/5][742/938] Loss_D: 0.6682 Loss_G: 1.4235 D(x): 0.6072 D(G(z)): 0.0581 / 0.3202
[4/5][743/938] Loss_D: 0.9379 Loss_G: 7.4046 D(x): 0.9936 D(G(z)): 0.5179 / 0.0010
[4/5][744/938] Loss_D: 1.1616 Loss_G: 1.4892 D(x): 0.3814 D(G(z)): 0.0059 / 0.2985
[4/5][745/938] Loss_D: 0.9120 Loss_G: 4.5745 D(x): 0.9924 D(G(z)): 0.4853 / 0.0201
[4/5][746/938] Loss_D: 0.3917 Loss_G: 4.2189 D(x): 0.8287 D(G(z)): 0.1436 / 0.0262
[4/5][747/938] Loss_D: 0.7902 Loss_G: 1.4401 D(x): 0.6064 D(G(z)): 0.1607 / 0.2988
[4/5][748/938] Loss_D: 0.6602 Loss_G: 6.1712 D(x): 0.9193 D(G(z)): 0.3692 / 0.0036
[4/5][749/938] Loss_D: 0.5375 Loss_G: 3.1360 D(x): 0.6442 D(G(z)): 0.0255 / 0.0817
[4/5][750/938] Loss_D: 0.2077 Loss_G: 3.3836 D(x): 0.9431 D(G(z)): 0.1264 / 0.0541
[4/5

[4/5][838/938] Loss_D: 0.6491 Loss_G: 2.1916 D(x): 0.7570 D(G(z)): 0.2614 / 0.1409
[4/5][839/938] Loss_D: 0.5720 Loss_G: 1.9428 D(x): 0.7344 D(G(z)): 0.2026 / 0.1724
[4/5][840/938] Loss_D: 0.6145 Loss_G: 1.5573 D(x): 0.7111 D(G(z)): 0.1858 / 0.2552
[4/5][841/938] Loss_D: 0.5971 Loss_G: 3.0354 D(x): 0.8890 D(G(z)): 0.3504 / 0.0620
[4/5][842/938] Loss_D: 0.4680 Loss_G: 2.7271 D(x): 0.7727 D(G(z)): 0.1534 / 0.0783
[4/5][843/938] Loss_D: 0.4240 Loss_G: 1.9125 D(x): 0.7586 D(G(z)): 0.1084 / 0.1799
[4/5][844/938] Loss_D: 0.5754 Loss_G: 3.3348 D(x): 0.8925 D(G(z)): 0.3371 / 0.0448
[4/5][845/938] Loss_D: 0.4093 Loss_G: 2.6447 D(x): 0.7706 D(G(z)): 0.1001 / 0.0957
[4/5][846/938] Loss_D: 0.3545 Loss_G: 2.5123 D(x): 0.8493 D(G(z)): 0.1483 / 0.1085
[4/5][847/938] Loss_D: 0.2090 Loss_G: 3.4870 D(x): 0.9458 D(G(z)): 0.1322 / 0.0437
[4/5][848/938] Loss_D: 0.1388 Loss_G: 3.6605 D(x): 0.9381 D(G(z)): 0.0685 / 0.0348
[4/5][849/938] Loss_D: 0.3488 Loss_G: 3.6111 D(x): 0.9020 D(G(z)): 0.1955 / 0.0342
[4/5